# Stock Market Predictions using Sentiment Scores
- **Devendra Govil**
- **Akshay Pamnani**

In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions
from pyspark.ml.feature import Imputer, OneHotEncoder, StandardScaler, VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

## Process Reddit Data

In [0]:
ss = SparkSession.builder.getOrCreate()

In [0]:
# REF_STRING = "mongodb+srv://admin:<password>@msds697-cluster.qzgwq.mongodb.net/"
# Credit: Taken from Teammates Youshi and Guru
def read_df_from_mongo(spark, collection_name):
    mongo_username = 'admin'
    mongo_password =  'msds697'
    mongo_ip_address = 'msds697-cluster.qzgwq.mongodb.net/'
    MONGO_DB_NAME = "msds697_project"
    connection_string = f"mongodb+srv://{mongo_username}:{mongo_password}@{mongo_ip_address}{MONGO_DB_NAME}.{collection_name}"
    spark_df = spark.read.format("mongo").option("uri", connection_string).load()
    return spark_df
    
def store_df_to_mongo(spark, spark_df, collection_name):
    mongo_username = 'admin'
    mongo_password =  'msds697'
    mongo_ip_address = 'msds697-cluster.qzgwq.mongodb.net/'
    MONGO_DB_NAME = "msds697_project"
    connection_string = f"mongodb+srv://{mongo_username}:{mongo_password}@{mongo_ip_address}{MONGO_DB_NAME}.{collection_name}"
    spark_df.write.format("com.mongodb.spark.sql.DefaultSource")\
                     .mode("append")\
                     .option("uri", connection_string)\
                     .save()

In [0]:
df_snp_500 = read_df_from_mongo(ss, 'snp_500_data')

In [0]:
df_snp_500.show()

+------------+-----------+---------+----------+--------------+----------+--------------+-----------+--------------------+--------------------+---------------------+------------+--------------------+------+----------+--------------+-----------+--------------------+--------------------+--------------------+------+--------------------+---------------+
52 Week High|52 Week Low| CIK|Date added|Dividend Yield| EBITDA|Earnings/Share| Founded| GICS Sector| GICS Sub-Industry|Headquarters Location| Market Cap| Name| Price|Price/Book|Price/Earnings|Price/Sales| SEC Filings| Sector| Security|Symbol| _id| state|
+------------+-----------+---------+----------+--------------+----------+--------------+-----------+--------------------+--------------------+---------------------+------------+--------------------+------+----------+--------------+-----------+--------------------+--------------------+--------------------+------+--------------------+---------------+
 175.49| 259.77| 66740.0|1957-03-04| 2.3328617| 9.048E9| 7.92| 1902| Industrials|Industrial Conglo...| Saint Paul, Minne...|138721055226| 3M Company|222.89| 11.34| 24.31| 4.3902707|http://www.sec.go...| Industrials| 3M| MMM|[640bcd5fc23625a7...| Minnesota|
 48.925| 68.39| 91142.0|2017-07-26| 1.1479592| 6.01E8| 1.7| 1916| Industrials| Building Products| Milwaukee, Wisconsin| 10783419933| A.O. Smith Corp| 60.24| 6.35| 27.76| 3.5754826|http://www.sec.go...| Industrials| A. O. Smith| AOS|[640bcd5fc23625a7...| Wisconsin|
 42.28| 64.6| 1800.0|1957-03-04| 1.9089824| 5.744E9| 0.26| 1888| Health Care|Health Care Equip...| North Chicago, Il...|102121042306| Abbott Laboratories| 56.27| 3.19| 22.51| 3.7404804|http://www.sec.go...| Health Care| Abbott| ABT|[640bcd5fc23625a7...| Illinois|
 60.05| 125.86|1551152.0|2012-12-31| 2.4995599| 1.031E10| 3.29|2013 (1888)| Health Care| Pharmaceuticals| North Chicago, Il...|181386347059| AbbVie Inc.|108.48| 26.14| 19.41| 6.291571|http://www.sec.go...| Health Care| AbbVie| ABBV|[640bcd5fc23625a7...| Illinois|
 114.82| 162.6|1467373.0|2011-07-06| 1.7144699|5.643228E9| 5.44| 1989|Information Techn...|IT Consulting & O...| Dublin, Ireland| 98765855553| Accenture plc|150.51| 10.62| 25.47| 2.604117|http://www.sec.go...|Information Techn...| Accenture| ACN|[640bcd5fc23625a7...| Ireland|
 38.93| 74.945| 718877.0|2015-08-31| 0.43190324| 2.704E9| 1.28| 2008|Communication Ser...|Interactive Home ...| Santa Monica, Cal...| 52518668144| Activision Blizzard| 65.83| 5.16| 31.8| 10.59512|http://www.sec.go...|Information Techn...| Activision Blizzard| ATVI|[640bcd5fc23625a7...| California|
 114.451| 204.45| 796343.0|1997-05-05| 0.0| 2.53804E9| 3.39| 1982|Information Techn...|Application Software| San Jose, California| 94550214268| Adobe Systems Inc|185.16| 11.06| 52.31| 13.092818|http://www.sec.go...|Information Techn...| Adobe Inc.| ADBE|[640bcd5fc23625a7...| California|
 78.81| 169.55|1158449.0|2015-07-09| 0.21832074| 8.53941E8| 6.19| 1932|Consumer Discreti...| Automotive Retail| Raleigh, North Ca...| 8123611867| Advance Auto Parts|109.63| 2.51| 19.54| 1.1301061|http://www.sec.go...|Consumer Discreti...| Advance Auto Parts| AAP|[640bcd5fc23625a7...| North Carolina|
 9.7| 15.65| 2488.0|2017-03-20| 0.0| 3.39E8| 0.03| 1969|Information Techn...| Semiconductors| Santa Clara, Cali...| 11191663795|Advanced Micro De...| 11.22| 21.47| 187.0| 2.1091955|http://www.sec.go...|Information Techn...| AMD| AMD|[640bcd5fc23625a7...| California|
 10.0| 12.05| 874761.0|1998-10-02| 4.961832| 3.001E9| -1.72| 1981| Utilities|Independent Power...| Arlington, Virginia| 6920851212| AES Corp| 10.06| 2.2| 9.96| 0.65951383|http://www.sec.go...| Utilities| AES Corporation| AES|[640bcd5fc23625a7...| Virginia|
 68.8| 91.73| 4977.0|1999-05-28| 2.4299066| 0.0| 11.01| 1955| Financials|Life & Health Ins...| Columbus, Georgia| 33422948000| AFLAC Inc| 83.25| 1.53| 12.24| 1.5429344|http://www.sec.go...| Financials| Aflac| AFL|[640bcd5fc23625a7...| Georgia|
 49.23| 75.0|1090872.0|2000-06-05| 0.8756979| 1.094E9| 2.1|

In [0]:
df_snp_500.distinct().count()

Out[146]: 397

In [0]:
df_sentiment_target = read_df_from_mongo(ss, 'sentiment_target_data')

In [0]:
df_reddit_sentiments = read_df_from_mongo(ss, 'reddit_post_sentiment_to_tickers')

In [0]:
df_reddit_sentiments.show(5)

+------+--------------------+-------------------------+------------------------+-------------------------+------------+
Symbol| _id|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit|ticker_small|
+------+--------------------+-------------------------+------------------------+-------------------------+------------+
 GOOGL|[640bf672c23625a7...| 0.2134772323575154| 0.6720852583247926| 0.1144407541255434| googl|
 GOOG|[640bf672c23625a7...| 0.207556259084261| 0.6761723574404802| 0.1162736969260437| goog|
 AMZN|[640bf672c23625a7...| 0.210345103778072| 0.6505878373076498| 0.1390899470978719| amzn|
 AAPL|[640bf672c23625a7...| 0.2889614963273045| 0.559575176497462| 0.151452323479381| aapl|
 EXPE|[640bf672c23625a7...| 0.225889461640842| 0.5445914479612926| 0.2295154382004115| expe|
+------+--------------------+-------------------------+------------------------+-------------------------+------------+
only showing top 5 rows

In [0]:
df_sentiment_target.show(5)

+--------------------+--------------------+--------+-----------+---------------+-----------+--------------------------+-----------------------+--------------------------+------+--------------+------------+------+
 Company| _id| cik|final_close|fiscal_year_end|monday_open|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score| sic|state_location|state_of_inc|ticker|
+--------------------+--------------------+--------+-----------+---------------+-----------+--------------------------+-----------------------+--------------------------+------+--------------+------------+------+
 Apple Inc.|[640beabbc23625a7...|320193.0| 150.59| 924.0| 153.785| 0.4688902127369219| 0.238821110110847| 0.292288677152231|3571.0| CA| CA| AAPL|
 MICROSOFT CORP|[640beabbc23625a7...|789019.0| 252.32| 630.0| 256.425| 0.3269895316147798| 0.2921125628285875| 0.3808979055566326|7372.0| WA| WA| MSFT|
UNITEDHEALTH GROU...|[640beabbc23625a7...|731766.0| 464.91| 1231.0| 481.37| 0.2604422079425404| 0.203394956662383| 0.5361628353950765|6324.0| MN| DE| UNH|
 JOHNSON & JOHNSON|[640beabbc23625a7...|200406.0| 151.24| 102.0| 154.66| 0.3647347721282167| 0.2399994802068532| 0.39526574766493|2834.0| NJ| NJ| JNJ|
 JOHNSON & JOHNSON|[640beabbc23625a7...|200406.0| 151.24| 101.0| 154.66| 0.6183592572304637| 0.1866230048135942| 0.195017737955942|2834.0| NJ| NJ| JNJ|
+--------------------+--------------------+--------+-----------+---------------+-----------+--------------------------+-----------------------+--------------------------+------+--------------+------------+------+
only showing top 5 rows

In [0]:
df_snp_500.createOrReplaceTempView("df_snp")
df_sentiment_target.createOrReplaceTempView("df_senti_target")

In [0]:
model_table = spark.sql("""
    SELECT *
    FROM df_snp
    INNER JOIN df_senti_target
    ON df_snp.Symbol = df_senti_target.ticker
""")


In [0]:
model_table2 = model_table.dropDuplicates(["Symbol"])

In [0]:
model_table2.createOrReplaceTempView("model_table2")

In [0]:
column_names = spark.sql("SELECT * FROM model_table2 LIMIT 0").columns


In [0]:
column_names

Out[156]: ['52 Week High',
 '52 Week Low',
 'CIK',
 'Date added',
 'Dividend Yield',
 'EBITDA',
 'Earnings/Share',
 'Founded',
 'GICS Sector',
 'GICS Sub-Industry',
 'Headquarters Location',
 'Market Cap',
 'Name',
 'Price',
 'Price/Book',
 'Price/Earnings',
 'Price/Sales',
 'SEC Filings',
 'Sector',
 'Security',
 'Symbol',
 '_id',
 'state',
 'Company',
 '_id',
 'cik',
 'final_close',
 'fiscal_year_end',
 'monday_open',
 'negativity_sentiment_score',
 'neutral_sentiment_score',
 'positivity_sentiment_score',
 'sic',
 'state_location',
 'state_of_inc',
 'ticker']

In [0]:
df_reddit_sentiments.show(5)

+------+--------------------+-------------------------+------------------------+-------------------------+------------+
Symbol| _id|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit|ticker_small|
+------+--------------------+-------------------------+------------------------+-------------------------+------------+
 GOOGL|[640bf672c23625a7...| 0.2134772323575154| 0.6720852583247926| 0.1144407541255434| googl|
 GOOG|[640bf672c23625a7...| 0.207556259084261| 0.6761723574404802| 0.1162736969260437| goog|
 AMZN|[640bf672c23625a7...| 0.210345103778072| 0.6505878373076498| 0.1390899470978719| amzn|
 AAPL|[640bf672c23625a7...| 0.2889614963273045| 0.559575176497462| 0.151452323479381| aapl|
 EXPE|[640bf672c23625a7...| 0.225889461640842| 0.5445914479612926| 0.2295154382004115| expe|
+------+--------------------+-------------------------+------------------------+-------------------------+------------+
only showing top 5 rows

In [0]:
all_data = model_table2.join(df_reddit_sentiments, model_table2["ticker"]==df_reddit_sentiments["Symbol"], "left_outer")

In [0]:
all_data.count()

Out[159]: 370

In [0]:
all_data.columns

Out[160]: ['52 Week High',
 '52 Week Low',
 'CIK',
 'Date added',
 'Dividend Yield',
 'EBITDA',
 'Earnings/Share',
 'Founded',
 'GICS Sector',
 'GICS Sub-Industry',
 'Headquarters Location',
 'Market Cap',
 'Name',
 'Price',
 'Price/Book',
 'Price/Earnings',
 'Price/Sales',
 'SEC Filings',
 'Sector',
 'Security',
 'Symbol',
 '_id',
 'state',
 'Company',
 '_id',
 'cik',
 'final_close',
 'fiscal_year_end',
 'monday_open',
 'negativity_sentiment_score',
 'neutral_sentiment_score',
 'positivity_sentiment_score',
 'sic',
 'state_location',
 'state_of_inc',
 'ticker',
 'Symbol',
 '_id',
 'negative_sentiment_reddit',
 'neutral_sentiment_reddit',
 'positive_sentiment_reddit',
 'ticker_small']

In [0]:
all_data2 = all_data.withColumn("label", ((100*(all_data["final_close"]- all_data["monday_open"]))/all_data["monday_open"]))

In [0]:
all_data2.show(5)

+------------+-----------+---------+----------+--------------+---------+--------------+-------+--------------------+--------------------+---------------------+------------+--------------------+------+----------+--------------+-----------+--------------------+--------------------+--------------------+------+--------------------+---------------+--------------------+--------------------+---------+-----------+---------------+-----------+--------------------------+-----------------------+--------------------------+------+--------------+------------+------+------+--------------------+-------------------------+------------------------+-------------------------+------------+-------------------+
52 Week High|52 Week Low| CIK|Date added|Dividend Yield| EBITDA|Earnings/Share|Founded| GICS Sector| GICS Sub-Industry|Headquarters Location| Market Cap| Name| Price|Price/Book|Price/Earnings|Price/Sales| SEC Filings| Sector| Security|Symbol| _id| state| Company| _id| cik|final_close|fiscal_year_end|monday_open|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score| sic|state_location|state_of_inc|ticker|Symbol| _id|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit|ticker_small| label|
+------------+-----------+---------+----------+--------------+---------+--------------+-------+--------------------+--------------------+---------------------+------------+--------------------+------+----------+--------------+-----------+--------------------+--------------------+--------------------+------+--------------------+---------------+--------------------+--------------------+---------+-----------+---------------+-----------+--------------------------+-----------------------+--------------------------+------+--------------+------------+------+------+--------------------+-------------------------+------------------------+-------------------------+------------+-------------------+
 49.23| 75.0|1090872.0|2000-06-05| 0.8756979| 1.094E9| 2.1| 1999| Health Care|Health Care Equip...| Santa Clara, Cali...| 21984606918|Agilent Technolog...| 65.05| 4.56| 27.45| 6.493563|http://www.sec.go...| Health Care|Agilent Technologies| A|[640bcd5fc23625a7...| California|AGILENT TECHNOLOG...|[640beabbc23625a7...|1090872.0| 138.93| 1031.0| 143.35| 0.261793969467507| 0.1582571388677718| 0.5799488916647212|3826.0| CA| DE| A| A|[640bf672c23625a7...| 0.1658935546875| 0.78662109375| 0.047637939453125| a|-3.0833623997209543|
 78.81| 169.55|1158449.0|2015-07-09| 0.21832074|8.53941E8| 6.19| 1932|Consumer Discreti...| Automotive Retail| Raleigh, North Ca...| 8123611867| Advance Auto Parts|109.63| 2.51| 19.54| 1.1301061|http://www.sec.go...|Consumer Discreti...| Advance Auto Parts| AAP|[640bcd5fc23625a7...| North Carolina|ADVANCE AUTO PART...|[640beabbc23625a7...|1158449.0| 129.83| 101.0| 139.04| 0.2099486315725147| 0.1385363615128607| 0.6515150069146245|5531.0| NC| DE| AAP| AAP|[640bf672c23625a7...| 0.2524464925130208| 0.608172098795573| 0.1393394470214843| aap| -6.623993095512069|
 131.12| 180.1| 320193.0|1982-11-30| 1.5795412|7.9386E10| 9.2| 1977|Information Techn...|Technology Hardwa...| Cupertino, Califo...|809508034020| Apple Inc.|155.15| 5.66| 16.86| 3.4586093|http://www.sec.go...|Information Techn...| Apple Inc.| AAPL|[640bcd5fc23625a7...| California| Apple Inc.|[640beabbc23625a7...| 320193.0| 150.59| 924.0| 153.785| 0.4688902127369219| 0.238821110110847| 0.292288677152231|3571.0| CA| CA| AAPL| AAPL|[640bf672c23625a7...| 0.2889614963273045| 0.559575176497462| 0.151452323479381| aapl|-2.0775758363949626|
 71.9| 106.27|1140859.0|2001-08-30| 1.6132456|9.91884E8| 1.64| 1985| Health Care|Health Care Distr...| Conshohocken, Pen...| 20587704101|AmerisourceBergen...| 91.55| 9.73| 15.54| 0.17396984|http://www.sec.go...| Health Care| AmerisourceBergen| ABC|[640bcd5fc23625a7...| Pennsylvania|AMERISOURCEBERGEN...|[640beabbc23625a7...|1140859.0| 150.0| 930.0| 155.51| 0.3714104358745563| 0.2688633097035041| 0.3597262544219397|5122.0| PA| DE| AB

In [0]:
all_data3 = all_data2.select(
#     Company Ticker
    df_snp_500["Symbol"],
    
#     Continuous Variables
    '52 Week High',
 '52 Week Low',
    'Dividend Yield',
 'EBITDA',
 'Earnings/Share',
        'Market Cap',
    'Price',
 'Price/Book',
 'Price/Earnings',
 'Price/Sales',

#     Sentiment Scores columns are below
    'negativity_sentiment_score',
 'neutral_sentiment_score',
 'positivity_sentiment_score',
    'negative_sentiment_reddit',
    'neutral_sentiment_reddit',
    'positive_sentiment_reddit',
    
#     target present below
    "label"
    
)

**To Note: Until now we have not cache our data, we cache our data now to avoid multiple re-runs of the dag right from the start. This should significantly boost our runs going forward**

In [0]:
all_data3.cache()

Out[164]: DataFrame[Symbol: string, 52 Week High: double, 52 Week Low: double, Dividend Yield: double, EBITDA: double, Earnings/Share: double, Market Cap: bigint, Price: double, Price/Book: double, Price/Earnings: double, Price/Sales: double, negativity_sentiment_score: double, neutral_sentiment_score: double, positivity_sentiment_score: double, negative_sentiment_reddit: double, neutral_sentiment_reddit: double, positive_sentiment_reddit: double, label: double]

In [0]:
all_data3.show(2)

+------+------------+-----------+--------------+--------+--------------+-----------+-----+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
Symbol|52 Week High|52 Week Low|Dividend Yield| EBITDA|Earnings/Share| Market Cap|Price|Price/Book|Price/Earnings|Price/Sales|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit| label|
+------+------------+-----------+--------------+--------+--------------+-----------+-----+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
 GIS| 49.65| 63.73| 3.5864594|3.1076E9| 2.77|31098243069|53.99| 7.26| 17.53| 2.0389178| 0.2166518387697363| 0.4241848822804558| 0.3591632789498079| 0.75543212890625| 0.1557769775390625| 0.0887947082519531|-0.9696511774335677|
 K| 58.76| 76.69| 3.363962| 1.827E9| 1.95|22182794875|65.98| 11.5| 16.5| 2.3173764| 0.480057255174804| 0.1680572263126097| 0.3518855185125862| null| null| null|-1.7016710102713466|
+------+------------+-----------+--------------+--------+--------------+-----------+-----+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
only showing top 2 rows

In [0]:
all_data3.show(1)

+------+------------+-----------+--------------+--------+--------------+-----------+-----+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
Symbol|52 Week High|52 Week Low|Dividend Yield| EBITDA|Earnings/Share| Market Cap|Price|Price/Book|Price/Earnings|Price/Sales|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit| label|
+------+------------+-----------+--------------+--------+--------------+-----------+-----+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
 GIS| 49.65| 63.73| 3.5864594|3.1076E9| 2.77|31098243069|53.99| 7.26| 17.53| 2.0389178| 0.2166518387697363| 0.4241848822804558| 0.3591632789498079| 0.75543212890625| 0.1557769775390625| 0.0887947082519531|-0.9696511774335677|
+------+------------+-----------+--------------+--------+--------------+-----------+-----+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
only showing top 1 row

**As can be screen from time stamps above, the first show command takes 3.03 seconds while the second (after the data has been cached) takes only 0.10 seconds**

In [0]:
all_data4 = all_data3.dropna(how='any', subset=['label']).cache()

In [0]:
all_data4.show(4)

+------+------------+-----------+--------------+----------+--------------+-----------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
Symbol|52 Week High|52 Week Low|Dividend Yield| EBITDA|Earnings/Share| Market Cap| Price|Price/Book|Price/Earnings|Price/Sales|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit| label|
+------+------------+-----------+--------------+----------+--------------+-----------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
 GIS| 49.65| 63.73| 3.5864594| 3.1076E9| 2.77|31098243069| 53.99| 7.26| 17.53| 2.0389178| 0.2166518387697363| 0.4241848822804558| 0.3591632789498079| 0.75543212890625| 0.1557769775390625| 0.0887947082519531|-0.9696511774335677|
 K| 58.76| 76.69| 3.363962| 1.827E9| 1.95|22182794875| 65.98| 11.5| 16.5| 2.3173764| 0.480057255174804| 0.1680572263126097| 0.3518855185125862| null| null| null|-1.7016710102713466|
 LEN| 43.647053| 72.17| 0.26268265|1.405319E9| 3.38|14615967194| 59.11| 1.76| 15.35| 1.1468264| 0.3736773838038573| 0.2727449482625361| 0.3535776679336065| null| null| null| 0.9109518935516894|
 AVY| 78.471| 123.67| 1.5682175| 8.312E8| 3.11|10104814319|110.77| 8.83| 22.11| 1.5206499| NaN| NaN| NaN| null| null| null| -4.767087505441876|
+------+------------+-----------+--------------+----------+--------------+-----------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+
only showing top 4 rows

## Feature Engineering approaches required for our dataset
- Fortunately we don't have any categorical variables, so we don't need to deal with them. 
- We need to perform Standard Scaling on all our numeric features 
- We need to impute missing values (in our data only for numeric features)
- We would also finally require vector assembler to get our features column

**Note: all feature engineering apporaches are fit using only the training data and the statistics/fit derived from training data is used to transform test data**

In [0]:
split_data = all_data4.randomSplit([0.8, 0.2])

In [0]:
train_data_og = split_data[0].cache()
test_data_og = split_data[1].cache()

**To Note: We use cache here not only to get better performance, but also to make sure that we get consistent results, as if we were not to cache it, the random splitting while re-computing the dag may result in a different split. Plus, iterative algorithms can also lead to multiple re-computations of the dag. Hence, caching is very important at this step.**

First fitting all feature engineering/imputation techniques on train data

In [0]:
all_data4.columns

Out[226]: ['Symbol',
 '52 Week High',
 '52 Week Low',
 'Dividend Yield',
 'EBITDA',
 'Earnings/Share',
 'Market Cap',
 'Price',
 'Price/Book',
 'Price/Earnings',
 'Price/Sales',
 'negativity_sentiment_score',
 'neutral_sentiment_score',
 'positivity_sentiment_score',
 'negative_sentiment_reddit',
 'neutral_sentiment_reddit',
 'positive_sentiment_reddit',
 'label']

# Fitting Model on all columns (including both sentiment scores)

In [0]:
cols_to_impute = [
    'negativity_sentiment_score',
     'neutral_sentiment_score',
     'positivity_sentiment_score',
     'negative_sentiment_reddit',
     'neutral_sentiment_reddit',
     'positive_sentiment_reddit',
    'Price/Book'
]
cols_to_scale = [
    '52 Week High',
     '52 Week Low',
     'Dividend Yield',
     'EBITDA',
     'Earnings/Share',
     'Market Cap',
     'Price',
     'Price/Book',
     'Price/Earnings',
     'Price/Sales',
    'negativity_sentiment_score',
     'neutral_sentiment_score',
     'positivity_sentiment_score',
     'negative_sentiment_reddit',
     'neutral_sentiment_reddit',
     'positive_sentiment_reddit',
]

In [0]:
train_data = train_data_og
test_data = test_data_og

In [0]:
imputer = Imputer(inputCols=cols_to_impute, outputCols=cols_to_impute).fit(train_data)


In [0]:
train_data  = imputer.transform(train_data)

In [0]:
va = VectorAssembler(inputCols=cols_to_scale, outputCol='assembled_col')

In [0]:
train_data = va.transform(train_data).cache()

In [0]:
std_scaler = StandardScaler(inputCol='assembled_col', outputCol='features').fit(train_data)

In [0]:
train_data = std_scaler.transform(train_data).cache()

In [0]:
train_data.show(4)

+------+------------+-----------+--------------+----------+--------------+------------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+
Symbol|52 Week High|52 Week Low|Dividend Yield| EBITDA|Earnings/Share| Market Cap| Price|Price/Book|Price/Earnings|Price/Sales|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit| label| assembled_col| features|
+------+------------+-----------+--------------+----------+--------------+------------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+
 GIS| 49.65| 63.73| 3.5864594| 3.1076E9| 2.77| 31098243069| 53.99| 7.26| 17.53| 2.0389178| 0.2166518387697363| 0.4241848822804558| 0.3591632789498079| 0.75543212890625| 0.1557769775390625| 0.0887947082519531|-0.9696511774335677|[49.65,63.73,3.58...|[0.56616334906701...|
 K| 58.76| 76.69| 3.363962| 1.827E9| 1.95| 22182794875| 65.98| 11.5| 16.5| 2.3173764| 0.480057255174804| 0.1680572263126097| 0.3518855185125862| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113|-1.7016710102713466|[58.76,76.69,3.36...|[0.67004548622714...|
 LEN| 43.647053| 72.17| 0.26268265|1.405319E9| 3.38| 14615967194| 59.11| 1.76| 15.35| 1.1468264| 0.3736773838038573| 0.2727449482625361| 0.3535776679336065| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113| 0.9109518935516894|[43.647053,72.17,...|[0.49771121255559...|
 MMM| 175.49| 259.77| 2.3328617| 9.048E9| 7.92|138721055226|222.89| 11.34| 24.31| 4.3902707| 0.6454165539350912| 0.1990560844514493| 0.1555273616134595| 0.2417488980455463| 0.5938057776211071| 0.1644418418001966| -5.27827648114902|[175.49,259.77,2....|[2.00112801868621...|
+------+------------+-----------+--------------+----------+--------------+------------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+
only showing top 4 rows

In [0]:
test_data = imputer.transform(test_data)
test_data = va.transform(test_data)
test_data = std_scaler.transform(test_data).cache()

In [0]:
test_data.show(4)

+------+------------+-----------+--------------+---------+--------------+-----------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+
Symbol|52 Week High|52 Week Low|Dividend Yield| EBITDA|Earnings/Share| Market Cap| Price|Price/Book|Price/Earnings|Price/Sales|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit| label| assembled_col| features|
+------+------------+-----------+--------------+---------+--------------+-----------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+
 AVY| 78.471| 123.67| 1.5682175| 8.312E8| 3.11|10104814319|110.77| 8.83| 22.11| 1.5206499| 0.4555936999889994| 0.21454283043210556| 0.3298634695788955| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113| -4.767087505441876|[78.471,123.67,1....|[0.89481176565232...|
 RF| 13.0| 19.9| 1.8987342| 0.0| 1.0|21500640000| 17.9| 1.32| 17.9| 3.474771| 0.5193828883696465| 0.2512492766351195| 0.229367834995234| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113|-11.709401709401703|[13.0,19.9,1.8987...|[0.14824015182016...|
 IRM| 32.05| 41.53| 7.08258|1.11614E9| 0.46| 9410249279| 32.07| 4.97| 15.42| 3.2898626| 0.4394863723668106| 0.2729123675284021| 0.2876012601047873| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113|-3.5427319211102954|[32.05,41.53,7.08...|[0.36546898967971...|
 EMN| 76.02| 104.08| 2.2630835| 2.152E9| 10.12|14226830813| 93.57| 2.78| 12.28| 1.4904785| 0.617672196676142| 0.2347878252547485| 0.1475399780691094| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113| -5.144694533762062|[76.02,104.08,2.2...|[0.86686279548991...|
+------+------------+-----------+--------------+---------+--------------+-----------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+
only showing top 4 rows

In [0]:
lr_model = LinearRegression()
lr_model.setPredictionCol("pred_label")

Out[238]: LinearRegression_7bad84dc3f68

In [0]:
lr_model_fit = lr_model.fit(train_data)

In [0]:
lr_model_fit.summary.rootMeanSquaredError

Out[240]: 2.8210368397142984

In [0]:
final_pred = lr_model_fit.transform(test_data)

In [0]:
eval_test = lr_model_fit.evaluate(test_data)

In [0]:
eval_test.rootMeanSquaredError

Out[243]: 2.7174907173179945

In [0]:
rf = RandomForestRegressor(featuresCol="features", numTrees=400).fit(train_data)

In [0]:
train_rf = rf.transform(train_data)

In [0]:
train_rf.show(4)

+------+------------+-----------+--------------+----------+--------------+------------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+-------------------+
Symbol|52 Week High|52 Week Low|Dividend Yield| EBITDA|Earnings/Share| Market Cap| Price|Price/Book|Price/Earnings|Price/Sales|negativity_sentiment_score|neutral_sentiment_score|positivity_sentiment_score|negative_sentiment_reddit|neutral_sentiment_reddit|positive_sentiment_reddit| label| assembled_col| features| prediction|
+------+------------+-----------+--------------+----------+--------------+------------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+-------------------+
 GIS| 49.65| 63.73| 3.5864594| 3.1076E9| 2.77| 31098243069| 53.99| 7.26| 17.53| 2.0389178| 0.2166518387697363| 0.4241848822804558| 0.3591632789498079| 0.75543212890625| 0.1557769775390625| 0.0887947082519531|-0.9696511774335677|[49.65,63.73,3.58...|[0.56616334906701...|-2.5134733922477066|
 K| 58.76| 76.69| 3.363962| 1.827E9| 1.95| 22182794875| 65.98| 11.5| 16.5| 2.3173764| 0.480057255174804| 0.1680572263126097| 0.3518855185125862| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113|-1.7016710102713466|[58.76,76.69,3.36...|[0.67004548622714...| -2.556726526720269|
 LEN| 43.647053| 72.17| 0.26268265|1.405319E9| 3.38| 14615967194| 59.11| 1.76| 15.35| 1.1468264| 0.3736773838038573| 0.2727449482625361| 0.3535776679336065| 0.23262845270385352| 0.6041187092953174| 0.1632466599070113| 0.9109518935516894|[43.647053,72.17,...|[0.49771121255559...|-2.9925649961277974|
 MMM| 175.49| 259.77| 2.3328617| 9.048E9| 7.92|138721055226|222.89| 11.34| 24.31| 4.3902707| 0.6454165539350912| 0.1990560844514493| 0.1555273616134595| 0.2417488980455463| 0.5938057776211071| 0.1644418418001966| -5.27827648114902|[175.49,259.77,2....|[2.00112801868621...| -3.35126758068636|
+------+------------+-----------+--------------+----------+--------------+------------+------+----------+--------------+-----------+--------------------------+-----------------------+--------------------------+-------------------------+------------------------+-------------------------+-------------------+--------------------+--------------------+-------------------+
only showing top 4 rows

In [0]:
eval = RegressionEvaluator()

In [0]:
eval.evaluate(train_rf)

Out[248]: 1.878513286529792

In [0]:
test_rf = rf.transform(test_data)

In [0]:
eval.evaluate(test_rf)

Out[250]: 2.382978788467753

## Summary of observations:
1. While fitting the Linear Regression Model we get a RMSE of:
  - 2.82 on train
  - 2.717 on test
2. While fitting a Random Forest Regressor we get a RMSE of:
  - 1.87 on train
  - 2.38 on test
3. We can clearly see that Linear Regression Model doesn't overfit as the test performance actually increases, however RF regressor does overfit a little because the test performance is poorer than the train performance

# Future:
1. Our next steps include building a knowledge graph and using that to further improve our predictions
2. Iterate over and improve the sentiment scores further